#### O Desafio

<p><img alt="Colaboratory logo" height="99px" 
src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTpw-4GbEpC5PiordgozbjH7ILPI-X6NU_d7Q&usqp=CAU'
align="left" hspace="10px" vspace="0px" ></p>
<h1>Prever Vendas Futuras 
<p>Projeto final para o curso Coursera "Como ganhar um concurso de ciência de dados"</p>
</h1>
<hr />
<a href="https://www.kaggle.com/c/competitive-data-science-predict-future-sales/rules">Kaggle</a>


##### Descrição
Este desafio serve como projeto final para o curso Coursera "Como ganhar um concurso de ciência de dados" .

Nesta competição, você trabalhará com um conjunto de dados de série temporal desafiador que consiste em dados diários de vendas, gentilmente cedidos por uma das **maiores empresas de software da russa** - a **1C Company** . 

Pedimos que você preveja as vendas totais de cada produto e loja no próximo mês. Ao resolver esta competição, você poderá aplicar e aprimorar suas habilidades em ciência de dados.


<p><img alt="Colaboratory logo" height="49px" 
src='https://static.1c.ru/images/logo.png'
align="left" hspace="10px" vspace="0px" ></p>
<h1> 1C COMPANY
</h1>
Fundada em 1991, a 1C Company é especializada em desenvolvimento, distribuição, publicação e suporte de software para o mercado de massa. 
 <a href="https://1c.ru/eng/title.htm">site daa empresa</a>

##### Dados

Você recebe dados históricos de vendas diários. A tarefa é prever a quantidade total de produtos vendidos em cada loja para o conjunto de teste. Observe que a lista de lojas e produtos muda ligeiramente a cada mês. Criar um modelo robusto que possa lidar com tais situações é parte do desafio.

**Descrições de arquivo**
> * **sales_train.csv** - o conjunto de treinamento. Dados históricos diários de janeiro de 2013 a outubro de 2015.
> * **test.csv** - o conjunto de teste. Você precisa prever as vendas dessas lojas e produtos para novembro de 2015.
> * **sample_submission.csv** - um arquivo de envio de amostra no formato correto.
> * **items.csv** - informações complementares sobre os itens / produtos.
> * **item_categories.csv**  - informações complementares sobre as categorias de itens.
> * **shops.csv** - informações complementares sobre as lojas.

**Campos de dados**
> * **ID  - um Id que representa uma tupla (Loja, Item) dentro do conjunto de teste
> * **shop_id** - identificador único de uma loja
> * **item_id** - identificador único de um produto
> * **item_category_id** - identificador único da categoria do item
> * **item_cnt_day** - número de produtos vendidos. **Você está prevendo um valor mensal desta medida**  💥
> * **item_price** - preço atual de um item
> * **data**  - data no formato dd / mm / aaaa
> * **date_block_num** - um número de mês consecutivo, usado por conveniência. Janeiro de 2013 é 0, fevereiro de 2013 é 1, ..., outubro de 2015 é 33
> * **item_name**  - nome do item
> * **shop_name** - nome da loja
> * **item_category_name** - nome da categoria do item

##### Regras

>* **Limites de envio**: No máximo 5 inscrições por dia.

[Submissions](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/submissions)

#### Analises

##### Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
class Output:
    def clear(self):
        clear_output()
output = Output()
import gc
import os
def plot(d,t='area'):
  if t =='area':
    return pd.DataFrame(d).sort_index().plot.area()
  if t =='line':
    return pd.DataFrame(d).sort_index().plot.line()
  if t =='hist':
    return pd.DataFrame(d).sort_index().plot.hist()
  if t =='bar':
    return pd.DataFrame(d).sort_index().plot.bar()

def transpose(d,l=100):
  x = pd.DataFrame(d)
  return x.head(l).transpose()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={"figure.figsize": (10, 2)})
import gc

In [ ]:
_path = '/kaggle/input/competitive-data-science-predict-future-sales/'
item_categories =  pd.read_csv(_path+'/item_categories.csv')
items =  pd.read_csv(_path+'/items.csv')
sales_train =  pd.read_csv(_path+'/sales_train.csv')
submission =  pd.read_csv(_path+'/sample_submission.csv')
shops =  pd.read_csv(_path+'/shops.csv')
test =  pd.read_csv(_path+'/test.csv')

##### Traduzindo dados...

In [ ]:
!pip install -U textblob
from textblob import TextBlob
for i in range(len(shops)):
  text = shops.loc[i,"shop_name"]
  hi_blob = TextBlob(text)
  text_pt = hi_blob.translate(to='pt',from_lang='ru')
  shops.loc[i,"shop_name"] =str(text_pt)
output.clear()

In [ ]:
#item_categories =  pd.read_csv('./items.csv')
for i in range(len(item_categories)):
  text = item_categories.loc[i,"item_category_name"]
  hi_blob = TextBlob(u''+text)
  text_pt = hi_blob.translate(to='pt')
  item_categories.loc[i,"item_category_name"] =str(text_pt)
output.clear()

In [ ]:
d = items.drop_duplicates()
print("duplicate rows removed\t",len(items)-len(d))

##### Analysis

###### Overview

> **shops**

In [ ]:
shops.describe().transpose()

In [ ]:
#https://aaaanchakure.medium.com/data-visualization-a6dccf643fbb
shops.shop_id.plot.line()
plt.show()

In [ ]:
import urllib.request
import codecs
from bs4 import BeautifulSoup
wiki = 'https://pt.wikipedia.org/wiki/Lista_de_cidades_da_R%C3%BAssia_por_popula%C3%A7%C3%A3o'
page = urllib.request.urlopen(wiki)
soup = BeautifulSoup(page, 'html5lib')
table = soup.find('table', attrs={'class': 'wikitable'})
cities = []
tds = [row for row in table.findAll('tr')]
for td in tds:
  if len(td.findAll("td"))==4:
    cities.append([td.findAll("td")[1].find("a").text.strip().upper(),
                 td.findAll("td")[2].find("a").text.strip().upper(),
                 td.findAll("td")[3].find("i").text.replace('\xa0','').strip()])
#print(rows)


In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
output.clear()

In [ ]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import re
def fn_city(n):
  n if n.find('"')==-1 else n[0:n.find('"')].strip()
  n = re.sub('[^A-Za-z]+', ' ', n).strip().upper()
  n = n.replace("CENTRO","")
  n = n.replace("CENTRAL","")
  n = n.replace("COMERCIAL","")
  n = n.replace("COMERCIO","")
  n = n.replace("CENTRAL","")
  n = n.replace("N ","")
  n = n.replace("DE ","")
  n = n.replace("MEGA ","")
  n = n.replace("COMPLEXO E ","")
  if n.find("SPB")>=0:
    n = "Distrito Federal do Noroeste"
  else:
    if n.find("UFA")>=0:
      n = "Bascortostão"
    else:
      if n.find("MOSC")>=0:
        n = "distrito federal central"
      else:
        n = n.split(' ')[0]
        c = [m[1] for m in cities]
        n, s= process.extractBests(n, c)[0]  
  return n
shops["shop_shortname"] = shops.shop_name.transform(lambda n:fn_city(n))

shops["shop_city"] = ""
shops["shop_district"]=""
shops["shop_population"]=0

query = list(shops.shop_shortname.value_counts().index)
output.clear()
for q in query:
  print(q.lower())
  r, i = process.extractBests(q, cities)[0]
  shops.loc[(shops.shop_shortname==q),"shop_city"] = r[0]
  shops.loc[(shops.shop_shortname==q),"shop_district"]= r[1]
  shops.loc[(shops.shop_shortname==q),"shop_population"]= int(r[2])

In [ ]:
transpose(shops.shop_city.sort_values())

In [ ]:
shops["shop_category"] = shops.shop_name.str.split(" ").map( lambda x: x[1].replace("(","").replace(",","").replace(".","").replace('\"','').upper()[0:1] )
cats = list(shops.shop_category.value_counts().index)
shops.loc[(shops.shop_category.isin(cats[3:])),"shop_category"]='O'
transpose(shops.shop_category.value_counts())

In [ ]:
cols = [ 'shop_id', 'shop_city', 'shop_district',  'shop_category']
shops = shops[cols]
shops.shop_district = shops.shop_district.astype('category')
shops.shop_district = shops.shop_district.cat.codes
shops.shop_category = shops.shop_category.astype('category')
shops.shop_category = shops.shop_category.cat.codes
shops.shop_city = shops.shop_city.astype('category')
shops.shop_city = shops.shop_city.cat.codes
shops.head()

In [ ]:
shops.shop_city.sort_index().plot.hist()
plt.show()

**item_categories**

In [ ]:
item_categories.head(1)

In [ ]:
item_categories.describe().transpose()

In [ ]:
item_categories["item_category_type"] =  item_categories.item_category_name+'- '
item_categories["item_category_type"] = item_categories.item_category_type.transform(lambda c: c.split("-")[0].strip().upper() if c.find(" ")==-1 else c.split(" ")[0].strip().upper()  )
item_categories["item_category_type"] = item_categories.item_category_type.transform(lambda c: c if c.find("-")==-1 else c.split("-")[0].strip().upper()  )
categories = (item_categories["item_category_type"].value_counts().index)
item_categories.loc[(item_categories.item_category_type.isin(categories[8:])),"item_category_type"]='OUTROS'
transpose(item_categories.item_category_type.value_counts())

In [ ]:
item_categories = item_categories[['item_category_id','item_category_type']]
item_categories.item_category_type = item_categories.item_category_type.astype('category')
item_categories.item_category_type = item_categories.item_category_type.cat.codes
item_categories.head(1)

In [ ]:
item_categories.item_category_type.sort_index().plot.hist()
plt.show()

**items**

In [ ]:
items.head(1)

In [ ]:
items.describe().transpose()

In [ ]:
items.item_category_id.sort_index().plot.hist()
plt.show()

**items_categories**

In [ ]:
items_categories = items.merge(item_categories,how='left',left_on='item_category_id', right_on='item_category_id')

In [ ]:
items_categories.describe().transpose()

In [ ]:
items_categories = items_categories[["item_id","item_category_id","item_category_type"]]
items_categories.reset_index(drop=True, inplace=True)
items_categories.head(1)

In [ ]:
items_categories.item_category_type.sort_index().plot.hist()
plt.show()

**sales_train**

In [ ]:
sales_train.head(1)

In [ ]:
sales_train.describe().transpose()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
sales_train.item_price.sort_index().plot.hist(ax=ax1)
ax1.set_xlabel("item_price")
sales_train.item_cnt_day.sort_index().plot.hist(ax=ax2)
ax2.set_xlabel("item_cnt_day")
plt.show()

**sales**

In [ ]:
sales = sales_train.copy()
sales = items.merge(sales,how='right',left_on='item_id', right_on='item_id')
len(sales),len(sales_train),len(items)

In [ ]:
sales.head(1)

In [ ]:
sales = items_categories.merge(sales,how='right',on=["item_id","item_category_id"])
len(sales),len(sales_train),len(items_categories)

In [ ]:
#sales.drop(columns=['item_name_x','item_name_y'],inplace=True)
sales.reset_index(drop=True,inplace=True)
sales.head(1)

In [ ]:
sales.date_block_num.sort_index().plot.hist()

In [ ]:
import time
ts = time.time()
sales.date  = pd.to_datetime(sales.date,errors='coerce')
years = (sales.date.dt.year.value_counts().index)
months= (sales.date.dt.month.value_counts().index)
years
sequence = 0
for i,y in enumerate(years.sort_values()):
  for m in months.sort_values():
    if y==2013 and m ==1:
      sales.loc[(sales.date.dt.year==y)&(sales.date.dt.month==m),"date_block_num"]=0
    else:
      sequence+=1
      sales.loc[(sales.date.dt.year==y)&(sales.date.dt.month==m),"date_block_num"]=sequence
gc.collect()
time.time() - ts

In [ ]:
sales.date_block_num.sort_index().plot.hist()

In [ ]:
sales["dayofweek"]= sales.date.dt.dayofweek
sales["weekday"]= sales.date.dt.weekday
sales["quarter"]= sales.date.dt.quarter
sales["week"]= sales.date.dt.week
sales["month"]= sales.date.dt.month
#sales["year"]= sales.date.dt.year
sales["weekofyear"]= sales.date.dt.weekofyear
sales.head()

In [ ]:
transpose(sales.item_price.describe())

In [ ]:
sales = sales[(sales.item_price <107980)]
sales = sales[(sales.item_price >0)]

In [ ]:
transpose(sales.item_price.describe())

In [ ]:
transpose(sales.item_cnt_day.describe())

In [ ]:
sales = sales[(sales.item_cnt_day <22)]
#sales = sales[(sales.item_cnt_day >)]
transpose(sales.item_cnt_day.describe())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
sales.date_block_num.sort_index().plot.hist(ax=ax1)
ax1.set_xlabel("item_price")
sales.item_cnt_day.sort_index().plot.hist(ax=ax2)
ax2.set_xlabel("item_cnt_day")
plt.show()

In [ ]:
#print("#1",len(sales))
#sales = sales[(sales.item_price >0)]
#print("#2",len(sales))
#sales = sales[(sales.item_cnt_day >0)]
sales.reset_index(drop=True, inplace=True)
print("#3",len(sales))

###### **Pivot Table**

In [ ]:
gc.collect()

In [ ]:
sales.pivot_table(index=["shop_id"],values=["item_cnt_day"], aggfunc='sum')

In [ ]:
sales.pivot_table(index=["shop_id"],columns=["date_block_num"],values=["item_cnt_day"], aggfunc=['sum','mean'],fill_value=0,margins=True)


In [ ]:
pd.pivot_table(sales,index=["date_block_num","shop_id","item_category_id"],values=["item_cnt_day"],aggfunc=["sum","mean"])

In [ ]:
pd.pivot_table(sales,index=["date_block_num","shop_id","item_category_id","item_id"],values=["item_cnt_day"],aggfunc=["sum","mean"])

In [ ]:
pd.pivot_table(sales,index=["date_block_num","shop_id","item_category_id","item_id"],values=["item_cnt_day"],aggfunc=["sum","mean"])

In [ ]:
sales.head(1)

In [ ]:
pd.pivot_table(sales,index=["date_block_num","month"],values=["item_cnt_day"],aggfunc=["sum"]).head()

In [ ]:
if True:
  sales = sales[sales.item_price >=1]

  test.loc[test.shop_id == 0, 'shop_id'] = 57
  test.loc[test.shop_id == 0, 'shop_id'] = 57
  test.loc[test.shop_id == 1, 'shop_id'] = 57
  test.loc[test.shop_id == 1, 'shop_id'] = 57
  test.loc[test.shop_id == 58, 'shop_id'] = 57
  test.loc[test.shop_id == 58, 'shop_id'] = 57
  test.loc[test.shop_id == 18, 'shop_id'] = 17
  test.loc[test.shop_id == 18, 'shop_id'] = 17
  test.loc[test.shop_id == 30, 'shop_id'] = 31
  test.loc[test.shop_id == 30, 'shop_id'] = 31
  test.loc[test.shop_id == 40, 'shop_id'] = 41
  test.loc[test.shop_id == 40, 'shop_id'] = 41
  test.loc[test.shop_id == 39, 'shop_id'] = 41
  test.loc[test.shop_id == 39, 'shop_id'] = 41
  test.loc[test.shop_id == 10, 'shop_id'] = 11
  test.loc[test.shop_id == 10, 'shop_id'] = 11
  test.loc[test.shop_id == 52, 'shop_id'] = 53
  test.loc[test.shop_id == 52, 'shop_id'] = 53
  test.loc[test.shop_id == 6, 'shop_id'] = 8
  test.loc[test.shop_id == 6, 'shop_id'] = 8
  test.loc[test.shop_id == 7, 'shop_id'] = 8
  test.loc[test.shop_id == 7, 'shop_id'] = 8

  sales.loc[sales_train.shop_id == 0, 'shop_id'] = 57
  sales.loc[sales_train.shop_id == 0, 'shop_id'] = 57
  sales.loc[sales_train.shop_id == 1, 'shop_id'] = 57
  sales.loc[sales_train.shop_id == 1, 'shop_id'] = 57
  sales.loc[sales_train.shop_id == 58, 'shop_id'] = 57
  sales.loc[sales_train.shop_id == 58, 'shop_id'] = 57
  sales.loc[sales_train.shop_id == 18, 'shop_id'] = 17
  sales.loc[sales_train.shop_id == 18, 'shop_id'] = 17
  sales.loc[sales_train.shop_id == 30, 'shop_id'] = 31
  sales.loc[sales_train.shop_id == 30, 'shop_id'] = 31
  sales.loc[sales_train.shop_id == 40, 'shop_id'] = 41
  sales.loc[sales_train.shop_id == 40, 'shop_id'] = 41
  sales.loc[sales_train.shop_id == 39, 'shop_id'] = 41
  sales.loc[sales_train.shop_id == 39, 'shop_id'] = 41
  sales.loc[sales_train.shop_id == 10, 'shop_id'] = 11
  sales.loc[sales_train.shop_id == 10, 'shop_id'] = 11
  sales.loc[sales_train.shop_id == 52, 'shop_id'] = 53
  sales.loc[sales_train.shop_id == 52, 'shop_id'] = 53
  sales.loc[sales_train.shop_id == 6, 'shop_id'] = 8
  sales.loc[sales_train.shop_id == 6, 'shop_id'] = 8
  sales.loc[sales_train.shop_id == 7, 'shop_id'] = 8
  sales.loc[sales_train.shop_id == 7, 'shop_id'] = 8

  sales = sales[(sales.item_price < 100000)]
  sales = sales[sales.item_cnt_day < 1001]


pd.pivot_table(sales,index=["shop_id"],values=["item_price"],aggfunc=['mean','max','min',len]).head()

In [ ]:
#sales = sales[sales.item_price >=1]
pd.pivot_table(sales,index=["shop_id"],values=["item_cnt_day"],aggfunc=['mean','max','min','count']).head()

In [ ]:
sales.head(1)

In [ ]:
sales_pivot = sales.pivot_table(
    index=['shop_id', 'item_id','item_category_id','item_category_type'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc=['sum']
).reset_index()

sales_pivot.head(2)


In [ ]:
(31/9)#sales_pivot.head(1)


In [ ]:
sales_pivot.describe()

In [ ]:
sales.columns

In [ ]:
train = sales.copy()
keys= ['shop_id', 'item_id','item_category_id','item_category_type']

In [ ]:
train = train.merge(sales_pivot, how='left', on=keys).fillna(0)
train.reset_index(drop=True,inplace=True)
len(train)


In [ ]:
len(train)

In [ ]:
test.head(1)

In [ ]:
data = test.copy()
data = items.merge(data,how='right',left_on='item_id', right_on='item_id', sort=False)
data = items_categories.merge(data,how='right',on=["item_id","item_category_id"] ,sort=False)
data.sort_values(by="ID", ascending=True, inplace=True)
data = data[['shop_id', 'item_id','item_category_id','item_category_type']]
data.reset_index(drop=True,inplace=True)


In [ ]:
data.head()

In [ ]:
train.drop_duplicates(inplace=True, keep=False)
train.reset_index(drop=True,inplace=True)
train.head(1)

In [ ]:
gc.collect()

In [ ]:
train.drop_duplicates(inplace=True)
train.rename_axis(inplace=True)

In [ ]:
len(train),len(sales),len(data)

In [ ]:
train.head(1)

In [ ]:
columns = ['item_id',   'item_category_id', 'item_category_type',
                'item_name',                    'date_block_num',
                  'shop_id',         'item_price',       'item_cnt_day',
                'dayofweek',            'weekday',            'quarter',
                     'week',              'month',         'weekofyear']

In [ ]:
data = data.merge(sales_pivot, how='left', on=keys).fillna(0)
#data.reset_index(drop=True,inplace=True)
len(data)

In [ ]:
train.to_pickle('./a.pkl')
data.to_pickle('./b.pkl')
shops.to_pickle('./c.pkl')
sales_pivot.to_pickle('./d.pkl')
gc.collect()

In [ ]:
import pandas as pd
import numpy as np
import gc
train = pd.read_pickle('./a.pkl')
data= pd.read_pickle('./b.pkl')
shops= pd.read_pickle('./c.pkl')
#sales_pivot= pd.read_pickle('./d.pkl')

In [ ]:
gc.collect()

In [ ]:
data.head(1)

In [ ]:
gc.collect()

In [ ]:
print(data.item_id.head(10).values)
print(test.item_id.head(10).values)

In [ ]:
len(data),len(train),len(sales)


In [ ]:
columns = data.columns

In [ ]:
data_train = train[columns].copy()
data_train = data_train.merge(shops,how='left',left_on='shop_id', right_on='shop_id')
data_train.head()

In [ ]:
data = data.merge(shops,how='left',left_on='shop_id', right_on='shop_id')
data.head()

In [ ]:
data_train.shape,data.shape

In [ ]:
gc.collect()

In [ ]:
data.shape,data_train.shape

In [ ]:
columns = [                   'shop_id',                    'item_id',
                 'item_category_id',         'item_category_type',
                        'shop_city',              'shop_district',
                    'shop_category',
 ('sum', 'item_cnt_day', 0),  ('sum', 'item_cnt_day', 1),
        ('sum', 'item_cnt_day', 2),  ('sum', 'item_cnt_day', 3),
        ('sum', 'item_cnt_day', 4),  ('sum', 'item_cnt_day', 5),
        ('sum', 'item_cnt_day', 6),  ('sum', 'item_cnt_day', 7),
        ('sum', 'item_cnt_day', 8),  ('sum', 'item_cnt_day', 9),
       ('sum', 'item_cnt_day', 10), ('sum', 'item_cnt_day', 11),
       ('sum', 'item_cnt_day', 12), ('sum', 'item_cnt_day', 13),
       ('sum', 'item_cnt_day', 14), ('sum', 'item_cnt_day', 15),
       ('sum', 'item_cnt_day', 16), ('sum', 'item_cnt_day', 17),
       ('sum', 'item_cnt_day', 18), ('sum', 'item_cnt_day', 19),
       ('sum', 'item_cnt_day', 20), ('sum', 'item_cnt_day', 21),
       ('sum', 'item_cnt_day', 22), ('sum', 'item_cnt_day', 23),
       ('sum', 'item_cnt_day', 24), ('sum', 'item_cnt_day', 25),
       ('sum', 'item_cnt_day', 26), ('sum', 'item_cnt_day', 27),
       ('sum', 'item_cnt_day', 28), ('sum', 'item_cnt_day', 29),
       ('sum', 'item_cnt_day', 30), ('sum', 'item_cnt_day', 31),
       ('sum', 'item_cnt_day', 32), ('sum', 'item_cnt_day', 33),
       ('sum', 'item_cnt_day', 34), ('sum', 'item_cnt_day', 35)]

In [ ]:
#data.columns

In [ ]:
data_train = data_train[columns]
data = data[columns]

In [ ]:
len(data.values[:,:-2]),len(data.values[:, -2:-1]),len(data.values[:,1:-1])

In [ ]:
len(data_train.values[:,:-2]),len(data_train.values[:, -2:-1]),len(data_train.values[:,1:-1])

In [ ]:
x, y = data_train.values[:,:-2], data_train.values[:, -2:-1].ravel()
y = y.clip(0,20)
xtest = data.values[:,:-2]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(x, y,stratify=y, test_size=0.3)

In [ ]:
print("xtrain:\t",len(xtrain),"\t",xtrain.shape)
print("ytrain:\t",len(ytrain),"\t",ytrain.shape)
print("xvalid:\t",len(xvalid),"\t",xvalid.shape)
print("yvalid:\t",len(yvalid),"\t",yvalid.shape)
print("\nxtest:\t",len(xtest),"\t",xtest.shape)


In [ ]:
scaler = StandardScaler()
#scaler.fit(xtrain)
#xtrain=scaler.transform(xtrain)
#xvalid=scaler.transform(xvalid)
#xtest=scaler.transform(xtest)

In [ ]:
xtest[0,:5],xtrain[0,:5],xvalid[0,:5]

#### Feature Engineering

In [ ]:
gc.collect()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau,LearningRateScheduler
import os
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
keras.backend.clear_session()

xtrain = np.reshape(xtrain, (xtrain.shape[0],xtrain.shape[1], 1))
xvalid = np.reshape(xvalid, (xvalid.shape[0],xvalid.shape[1], 1))


def learning_rate_scheduler(self,epoch=0):
  lr = 1e-2
  print(' Learning rate: ', lr,' - epoch: ',epoch)
  return lr
batch_size = 512
unit = int(xtrain.shape[1])
model = keras.Sequential()
model.add(keras.layers.LSTM(unit, return_sequences=True, input_shape=(xtrain.shape[1], 1)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(unit, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.LSTM(unit))
model.add(keras.layers.Dense(1))
train_size = xtrain.shape[0]
model.compile(optimizer='RMSprop', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error', dtype=None)])

lr_scheduler = LearningRateScheduler(learning_rate_scheduler)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                            cooldown=1,
                            patience=2,
                            min_lr=0.5e-2)
callbacks = [ lr_scheduler, lr_reducer]

gc.collect()


In [ ]:
steps_per_epoch =  math.ceil(train_size / batch_size )
history = model.fit(xtrain, ytrain,  
                            epochs=5, verbose=1, 
                            steps_per_epoch=steps_per_epoch, 
                            batch_size=batch_size, 
                            validation_data=(xvalid, yvalid),
                            callbacks=callbacks)

In [ ]:
name="FRANCKE"
epochs = list(range(1,(np.array(history.history['root_mean_squared_error']).shape[0])+1))
plt.clf()
fig = plt.figure(figsize=(15, 2))
ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax.set_title(name+' - RMSE')
ax.plot(epochs,history.history['root_mean_squared_error'], "r.-")
ax.plot(epochs,history.history['val_root_mean_squared_error'], "c.-")
ax.legend(('RMSE','VAL RMSE'),fontsize=15)
ax.set_xlabel("Epochs")
ax.grid(True)

ax2.set_title(name+' - LOSS')
ax2.plot(epochs,history.history['loss'], "r.-")
ax2.plot(epochs,history.history['val_loss'], "c.-")
ax2.legend(('LOSS','VAL LOSS'),fontsize=15)
ax2.set_xlabel("Epochs")
ax2.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error
ypred = model.predict(xvalid)

In [ ]:
print('MSE =', mean_squared_error(yvalid, ypred, squared=False))
print('RMSE =', mean_squared_error(yvalid, ypred, squared=True))

In [ ]:
x = np.reshape(xtest, (xtest.shape[0],xtest.shape[1], 1))
preds = model.predict(x)

In [ ]:
ids = pd.read_csv(_path+'/test.csv')
submission = pd.DataFrame({
    "ID": ids.index, 
    "item_cnt_month": preds.reshape(-1).clip(0,20)
})
#https://pbpython.com/pandas-pivot-table-explained.html
submission.head(10)

In [ ]:
min(preds.reshape(-1)), np.median(preds.reshape(-1))

In [ ]:
submission.to_csv('francke_peixoto.csv', index=False)
